# Ranking school examination results using multivariate hierarchical models

In [12]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt

In [13]:
import json 

with open("data/data.json") as file:
    data = json.load(file)
    
with open("data/init.json") as file:
    init = json.load(file)
    
with open("data/params.json") as file:
    params = json.load(file)

In [14]:
for idx, item in data.items():
    data[idx] = np.array(item).reshape(-1)

for idx, item in init.items():
    init[idx] = np.array(item).reshape(-1)
    
for idx, item in params.items():
    params[idx] = np.array(item).reshape(-1)

In [15]:
data.keys(), init.keys(), params.keys()

(dict_keys(['Gender', 'LRT', 'M', 'N', 'R', 'school', 'School_denom', 'School_gender', 'Y', 'VR']),
 dict_keys(['theta', 'phi', 'gamma', 'beta', 'Omega', 'alpha']),
 dict_keys(['R_shape', 'School_denom_shape', 'School_gender_shape', 'Venom_shape', 'Omega_shape', 'alpha_shape']))

In [16]:
params

{'R_shape': array([3, 3]),
 'School_denom_shape': array([1978,    3]),
 'School_gender_shape': array([1978,    2]),
 'Venom_shape': array([1978,    3]),
 'Omega_shape': array([3, 3]),
 'alpha_shape': array([38,  3])}

In [150]:
hyperparams = dict()
hyperparams["tau0"] = 1e-3

In [196]:
def sampler(n, data, init, params, hyperparams):
    
    # initialisation de la chaine 
    chain = dict()
    for key, param in init.items():
        chain[key] = np.zeros((n+1, param.shape[0]))
        chain[key][0, :] = param
    
    # définition des constantes
    Y = data["Y"]
    
    N   = Y.shape[0]
    n_beta = init["beta"].shape[0]
    n_schools = np.max(data["school"])
    
    LRT_sum = np.sum(data["LRT"])
    tau0 = hyperparams["tau0"]
    
    T0 = np.linalg.inv(init["Omega"].reshape(3, 3))
    alpha_idx = np.arange(init["alpha"].shape[0])
    
    # définition de v
    LRT = data["LRT"].reshape(-1, 1)
    VR = data["VR"].reshape(N, -1, order="F")
    Girl = data["Gender"].reshape(-1, 1)
    School_gender = data["School_gender"].reshape(N, -1, order="F")
    School_denom = data["School_denom"].reshape(N, -1, order="F")

    v = np.concatenate((LRT**2, VR[:, [1]], Girl , School_gender, School_denom), axis=1)
    
    # définition de u
    u = np.concatenate((np.ones((N, 1)), LRT, VR[:, [1]]), axis=1)
    
    # initialisation de mu et tau
    mu  = np.zeros(N) 
    
    # mise à jour des paramètres
    for i in range(1, n+1):
        # calcul de mu
        alpha_current = chain["alpha"][i-1, :]
        beta_current = chain["beta"][i-1, :].reshape(1, -1)
        for k in range(N):
            j = data["school"][k]
            alpha_j = alpha_current[(3*(j-1)):(3*j)].reshape(1, -1)
            mu[k] = np.dot(alpha_j, u[[k], :].T) + np.dot(beta_current, v[[k], :].T)
        beta_current = beta_current.flatten()
        
        # calcul de tau
        theta_current = chain["theta"][i-1, :]
        phi_current = chain["phi"][i-1, :]        
        tau = np.exp(theta_current + phi_current * LRT).flatten()
    
        # mise à jour de phi: mcmc 
        # phi peut prendre toutes les valeurs réelles, 
        # donc on utilise le "random walk M.-H. scheme"
        phi_proposal = phi_current + np.random.normal()
        
        phi_top = (-tau0/2) * np.power(phi_proposal, 2)
        phi_top += (LRT_sum/2) * phi_proposal
        phi_top -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_proposal*data["LRT"]) * np.power(data["Y"]-mu, 2)
        )
    
        phi_bottom = (-tau0/2) * np.power(phi_current, 2)
        phi_bottom += (LRT_sum/2) * phi_current
        phi_bottom -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu, 2)
        )
        
        phi_acceptance = np.exp(phi_top-phi_bottom)
        if phi_acceptance < np.random.rand():
            chain["phi"][i,:] = phi_proposal
            phi_current = phi_proposal
        else:
            chain["phi"][i,:] = phi_current

        # mise à jour de theta: mcmc 
        # theta peut prendre toutes les valeurs réelles, 
        # donc on utilise le "random walk M.-H. scheme"        
        theta_proposal = theta_current + np.random.normal()
        
        theta_top = (-tau0/2) * np.power(theta_proposal, 2)
        theta_top += (N/2) * theta_proposal
        theta_top -= (np.exp(theta_proposal)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu, 2)
        )
        
        theta_bottom = (-tau0/2) * np.power(theta_current, 2)
        theta_bottom += (LRT_sum/2) * theta_current
        theta_bottom -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu, 2)
        ) 
        
        theta_acceptance = np.exp(theta_top-theta_bottom)
        if theta_acceptance < np.random.rand():
            chain["theta"][i,:] = theta_proposal
            theta_current = theta_proposal
        else:
            chain["theta"][i,:] = theta_current
                      
        # mise à jour de beta: normale multivariée        
        beta_cov_inv = tau0 * np.eye(n_beta) 
        for k in range(N):
            beta_cov_inv = beta_cov_inv + tau[k] * np.dot(v[[k], :].T, v[[k], :])
        beta_cov = np.linalg.inv(beta_cov_inv)
        
        beta_mean = np.sum(tau * Y * v.T, axis=1, keepdims=True)
        for j in range(1, n_schools+1):
            school_j = (data["school"] ==j)
            alpha_j = alpha_current[(3*(j-1)):(3*j)].reshape(1, -1)
            beta_mean = beta_mean - np.dot(alpha_j, np.dot(u[school_j, :].T, v[school_j, :])).T
        beta_mean = np.dot(beta_cov, beta_mean).flatten()
        
        beta_proposal = np.random.multivariate_normal(beta_mean, beta_cov)
        
        chain["beta"][i, :] = beta_proposal
        beta_current = beta_proposal
        
        # mise à jour de gamma: normale
        T_current = np.linalg.inv(chain["Omega"][i-1, :].reshape(3, 3))

        alpha_current_sum = np.zeros(3)
        for j in range(1, n_schools+1):
            alpha_current_sum = alpha_current_sum + alpha_current[(3*(j-1)):(3*j)]
        alpha_current_sum = alpha_current_sum.reshape(-1, 1)
        
        gamma_cov = np.linalg.inv(T0+n_schools*T_current)
        gamma_mean = np.dot(gamma_cov, np.dot(T_current, alpha_current_sum))
        gamma_mean = gamma_mean.flatten()
        
        gamma_proposal = np.random.multivariate_normal(gamma_mean, gamma_cov)
        
        chain["gamma"][i, :] = gamma_proposal
        gamma_current = gamma_proposal
        
        # mise à jour de alpha: normale
        beta_current = beta_current.reshape(1, -1)
        for j in range(1, n_schools+1):
            school_j = (data["school"] == j)
            tau_j = tau[school_j]
            u_j = u[school_j, :]
            v_j = v[school_j, :]
            Y_j = Y[school_j]
            
            n_j = len(Y_j)
            alphaj_cov = T_current
            for k in range(n_j):
                u_kj =  u[[k], :]
                alphaj_cov = alphaj_cov + tau_j[k] * np.dot(u_kj.T, u_kj)
            
            alphaj_mean = np.dot(T_current, gamma_current.reshape(-1,1)).reshape(1,-1)
            for k in range(n_j):
                alphaj_mean = alphaj_mean + tau_j[k] * (Y_j[k] - np.dot(beta_current, v_j[[k], :].T)) * u[[k], :]
            alphaj_mean = alphaj_mean.flatten()
            
            alphaj_proposal = np.random.multivariate_normal(alphaj_mean, alphaj_cov)
            chain["alpha"][i,(3*(j-1)):(3*j)] = alphaj_proposal
            
        
        # mise à jour de T=inverse de Omega: mcmc
        chain["Omega"][i, :] = chain["Omega"][i-1, :]
    
    return chain

In [198]:
sampler(10, data, init, params, hyperparams)

/tmp/ipykernel_2749/2566905562.py:94: RuntimeWarning: overflow encountered in exp
  theta_acceptance = np.exp(theta_top-theta_bottom)
/tmp/ipykernel_2749/2566905562.py:70: RuntimeWarning: overflow encountered in exp
  phi_acceptance = np.exp(phi_top-phi_bottom)
/tmp/ipykernel_2749/2566905562.py:156: RuntimeWarning: covariance is not positive-semidefinite.
  alphaj_proposal = np.random.multivariate_normal(alphaj_mean, alphaj_cov)


{'theta': array([[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [1.62708052],
        [1.62708052],
        [1.93308197],
        [3.72783939],
        [4.0253692 ],
        [4.08839746]]),
 'phi': array([[0.01      ],
        [0.31847195],
        [1.18811167],
        [1.18811167],
        [1.24695779],
        [1.24695779],
        [2.43140589],
        [2.43140589],
        [2.43140589],
        [2.43140589],
        [2.43140589]]),
 'gamma': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.55195680e-01,  2.52588870e-01, -1.67985766e+00],
        [-3.40293642e-01,  1.04013653e+01, -6.45262086e-01],
        [-3.01852531e+03, -1.10271833e+04,  1.22691914e+02],
        [-1.34176398e+14, -6.76495355e+17, -6.32989361e+13],
        [-9.81365199e+19,  1.42355018e+22, -1.18195005e+20],
        [ 4.07383277e+24, -8.95687969e+26, -3.94292591e+23],
        [-2.86217641e+29,  5.96396566e+31,  1.85189394e+28],